In [1]:
import re

class Grammar:
    def __init__(self, rules):
        rules = tuple(rules)
        self.rules = tuple(self._parse(rule) for rule in rules)

    def _parse(self, rule):
        return tuple(rule.replace(' ', '').split('->'))
        
    def __getitem__(self, nonterminal):
        yield from [rule for rule in self.rules if rule[0] == nonterminal]
        
    @staticmethod #metodo da classe
    def is_nonterminal(symbol):
        return symbol.isalpha() and symbol.isupper()

    @property
    def nonterminals(self):
        return set([nt for nt, _ in self.rules])
        
    @property
    def terminals(self):       
        a = [
            expression
            for _, expression in self.rules
            if not self.is_nonterminal(expression)
        ]
        
        nt = sorted([nt for nt, _ in self.rules], key=len, reverse=True)

        for i in nt:
            for j in range(0, len(a)):
                if re.search(i, a[j]):
                    a[j] = a[j].replace(i, '')
                a[j] = " ".join(a[j].split())
        
        tmp = []  
        for i in a:
            if i != '' and i not in tmp:
                tmp.append(i)
        a = tmp

        tmp = []
        for i in a:
            if len(i) > 1 and not i.isalpha():
                tmp2 = []
                tmp2.append(list(i))
                tmp2 = [i for j in tmp2 for i in j]
                for j in tmp2:
                    tmp.append(j)
            else:
                tmp.append(i)
        a = tmp
        tmp = []  
        return set(a)

In [2]:
def transform2(string,lista):
    listanova=[]
    texto=""
    tam=len(string)
    x = 0
    while x < tam:
        texto +=string[x]
        if(x < tam-1 and texto+string[x+1] in lista):
            listanova.append(texto+string[x+1])
            texto=""
            x +=1
        elif texto in lista: 
            listanova.append(texto)
            texto=""
        x+=1
    return listanova

In [3]:
def union(first, begins):
    n = len(first)
    first |= begins
    return len(first) != n

In [4]:
def First(grammar):
    first = {i: set() for i in grammar.nonterminals}
    first.update((i, {i}) for i in grammar.terminals)
    lista = sorted((list(grammar.terminals)+list(grammar.nonterminals)), key=len, reverse=True)
    rule = tuple([(i, transform2(j, lista)) for i, j in grammar.rules])
    epsilon = set()

    while True:
        updated = False
        for nt, expression in rule:
            for symbol in expression:
                updated |= union(first[nt], first[symbol])
                if symbol not in epsilon:
                    break
                else:
                    updated |= union(epsilon, {nt})
        
        if not updated:
            epsilon = {'ε'}
            for i in first:
                if first[i] == set():
                    first[i] = {'ε'} 
            return first, epsilon

In [5]:
def tratarArq(lista): # trata lista e converte
    lista = " ".join(lista.split())
    lista = re.split(r' ', lista)
    return lista

In [6]:
def lerArquivo():
    arquivo = []
    b = []
    inicio = ""
    fim = ""
    
    with open("gramatica.txt", "r") as gramatica:
        for line in gramatica:
            arquivo.append(line.replace('Îµ', 'ε').strip().split('\n'))
    arquivo = [i for j in arquivo for i in j]

    for i in range(0, len(arquivo)):
        temp = []
        temp = tratarArq(arquivo[i])
        arquivo[i] = temp
    for i in range(0, len(arquivo)):
        tam = len(arquivo[i])
        for j in range(0, tam):    
            if "|" in arquivo[i][j]:
                inicio = arquivo[i][0]+" "+arquivo[i][1]
                fim = arquivo[i][-1]
                suma = inicio+" "+fim
                arquivo.append(tratarArq(suma))
    for i in arquivo:
        b.append(" ".join(i[:3]))
    return b

In [7]:
def main():
    gr = lerArquivo()
    a = Grammar(gr)
    first, epsilon = First(a)
    for i in first:
        print("Primeiro({})".format(i),"=", first[i])

In [8]:
main()

Primeiro(F) = {'(', 'id'}
Primeiro(T') = {'*', 'ε'}
Primeiro(E') = {'+', 'ε'}
Primeiro(E) = {'(', 'id'}
Primeiro(T) = {'(', 'id'}
Primeiro(() = {'('}
Primeiro(+) = {'+'}
Primeiro()) = {')'}
Primeiro(*) = {'*'}
Primeiro(id) = {'id'}
Primeiro(ε) = {'ε'}


In [9]:
# Logica 
# (('E', "TE'"), ("E'", "+TE'"), ('T', "FT'"), ("T'", "*FT'"), ('F', '(E)'), ("E'", 'ε'), ("T'", 'ε'), ('F', 'id'))

# [['E', "TE'"], ["E'", "+TE'"], ['T', "FT'"], ["T'", "*FT'"], ['F', '(E)'], ["E'", 'ε'], ["T'", 'ε'], ['F', 'id']]

# [['E', ["T","E'"]], ["E'", ["+","T","E'"]], ['T', ["F","T'"]], ["T'", ["*","F","T'"]], ['F', ["(","E",")"]], ["E'", ['ε']], ["T'", ['ε']], ['F', ['id']]]

# (('E', ["T","E'"]), ("E'", ["+","T","E'"]), ('T', ["F","T'"]), ("T'", ["*","F","T'"]), ('F', ["(","E",")"]), ("E'", ['ε']), ("T'", ['ε']), ('F', ['id']) )